In [0]:
%run /Repos/ruddys@sandiego.edu/Movie-Recommendation-system-using-Azure-and-Databricks/Analysis/Authorization

In [0]:
#import required libraries
import datetime
import pyspark.sql.functions as f
import pyspark.sql.types
import pandas as pd

from pyspark.sql.functions import year, month, dayofmonth
from pyspark.sql.functions import unix_timestamp, from_unixtime
from pyspark.sql import Window
from pyspark.sql.functions import rank, min

In [0]:
#test check to access the container
dbutils.fs.ls("abfss://containerdatalake@samovieanalysis.dfs.core.windows.net")

In [0]:
# list all the mounts points
dbutils.fs.mounts()

In [0]:
#Unzip the file Using the DBFS
#copy thie file into local system file
dbutils.fs.cp("dbfs:/mnt/movielens/ml_latest_small.zip", "file:/tmp/ml_latest_small.zip")



In [0]:
%sh
ls -l /tmp/ml_latest_small.zip

In [0]:
# %sh
# rm -r /tmp/ml_latest_small.zip/

In [0]:
%sh 
unzip /tmp/ml_latest_small.zip -d /tmp

In [0]:
# Now if we use the `ls` we will see all the files that already copued into the local file system.
dbutils.fs.ls("file:/tmp/ml-latest-small/")

In [0]:
# Copy the files back from the local file system into the actual container (Edge Lake Container)
dbutils.fs.cp("file:/tmp/ml-latest-small/movies.csv", "abfss://containerdatalake@samovieanalysis.dfs.core.windows.net/movies.csv") 
dbutils.fs.cp("file:/tmp/ml-latest-small/ratings.csv", "abfss://containerdatalake@samovieanalysis.dfs.core.windows.net/ratings.csv") 
dbutils.fs.cp("file:/tmp/ml-latest-small/tags.csv", "abfss://containerdatalake@samovieanalysis.dfs.core.windows.net/tags.csv") 
dbutils.fs.cp("file:/tmp/ml-latest-small/links.csv", "abfss://containerdatalake@samovieanalysis.dfs.core.windows.net/links.csv") 
